In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from  torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.models import resnet50

import numpy as np
np.random.seed(42)

import pickle
import pandas as pd
import os
from skimage.io import imread

import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from tqdm.notebook import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
# !pip3 install -U sentence-transformers

In [3]:
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
# from sentence_transformers import SentenceTransformer

In [4]:
image_dim = 224

def show_sample(sample):
    """
    Displays a sample as they come out of the trainloader.
    """

    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle(sample['caption'], size=20)
    ax1.imshow(sample['full_image'].permute(1,2,0))
    ax2.imshow(sample['masked_image'].permute(1,2,0))
    plt.show()

class COCODataset(Dataset):
    def __init__(self, annotations, datadir, transform=None):
        """
        Dataset of obfuscated coco images, with captions.
        
        annotations: load from pickle, akshay's processed annotations
        datadir: Preprocessed data. Contains /originals and /masked
        tranforms: function to be run on each sample
        """
        
        self.datadir = datadir
        self.transform = transform
        self.annotations = annotations
        self.filenames = os.listdir(datadir)
        
        # Since every 5 samples is the same image, we have a one image cache.
        # TODO this may get fucky with shuffle? we can find out later.
        self.last_image = None
        self.last_index = None
        
    def __len__(self):
        return len(self.filenames) * 5
    
    def __getitem__(self, idx):
        """
        Gets images from the dataset.
        
        Each image has 5 replicas, with different captions and sections
        
        Returns: dictionary with blanked out ['image'] and ['caption']
            image: FloatTensor
            caption: string (may later be a list)
        """

        # Load image or retrieve from cache
        
        image_filename = self.filenames[idx // 5]
        image_id = int(image_filename.split(".")[0])
        
        
        if self.last_index is not None and idx // 5 == self.last_index // 5:
            full_image = self.last_image
        else:
            image_filepath = os.path.join(self.datadir, image_filename)
            full_image = Image.open(image_filepath)
            self.last_image = full_image
        
        self.last_index = idx
        full_image = full_image.convert("RGB") # The occasional 1 channel grayscale image is in there.
        full_image = full_image.resize((image_dim, image_dim))

        # Fetch annotation, mask out area
        anno = self.annotations[image_id][idx % 5]
        
        masked_image = full_image.copy()
        
        draw = ImageDraw.Draw(masked_image)
        draw.rectangle([(anno['coord_start'][0], anno['coord_start'][1]), (anno['coord_end'][0], anno['coord_end'][1])], fill="black")

        sample = {'masked_image': masked_image, 'caption': anno['caption'], 'full_image': full_image, 'image_id':image_id}

        if self.transform:
            sample = self.transform(sample)
            
        return sample

In [5]:
%%time
annos = pd.read_pickle("../annotations_train2017.pickle")

# Recommended resnet transforms.
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
# TODO change masking logic to accomodate this
#resnet_transform = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), normalize, transforms.ToTensor()])
#resnet_transform = transforms.Compose([transforms.Resize((image_dim,image_dim)), transforms.ToTensor(), normalize])
resnet_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

def basic_transform_sample(sample):
    """
    A "default" transformer. Applies recommended resnet transforms.
    """
    sample['masked_image'] = resnet_transform(sample['masked_image'])
    sample['full_image'] = resnet_transform(sample['full_image'])
    return sample

dataset_train = COCODataset(annos, "../data/train2017", transform=basic_transform_sample)

CPU times: user 1.58 s, sys: 234 ms, total: 1.82 s
Wall time: 1.81 s


In [28]:
class rotfoNET_AE(nn.Module):
    def __init__(self):
        super(rotfoNET_AE, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=(5,5))
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2,2), return_indices=True)
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2,2), return_indices=True)
        self.unconv1 = nn.ConvTranspose2d(6,3,kernel_size=(5,5))
        self.maxunpool1 = nn.MaxUnpool2d(kernel_size=(2,2))
        self.unmaxunpool2 = nn.MaxUnpool2d(kernel_size=(2,2))
        
        self.encoder1 = nn.Sequential(
            nn.Tanh(),
            nn.Conv2d(6, 12,kernel_size=(5,5)),
        )
        
        self.encoder2 = nn.Sequential(
            nn.Tanh(),
            nn.Conv2d(12, 16, kernel_size=(5,5)),
            nn.Tanh()
        )
        
        self.decoder2 = nn.Sequential(
            nn.ConvTranspose2d(16, 12, kernel_size=(5,5)),
            nn.Tanh()
        )
        
        self.decoder1 = nn.Sequential(
            nn.ConvTranspose2d(12,6,kernel_size=(5,5)),
            nn.Tanh(),
        )
        

    def forward(self, x):
        x = self.conv1(x)
        x,indices1 = self.maxpool1(x)
        x = self.encoder1(x)
        x,indices2 = self.maxpool2(x)
        x = self.encoder2(x)
        
        x = self.decoder2(x)
        x = self.unmaxunpool2(x, indices2)
        x = self.decoder1(x)
        x = self.maxunpool1(x,indices1)
        x = self.unconv1(x)
        x = nn.Tanh()(x)
        return x


In [29]:
n_epoch = 10
batch_size = 64
lr = 1e-3

trainloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=False, num_workers=3) # VERY important to make sure num_workers > 0.

model = rotfoNET_AE().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
for epoch in tqdm(range(n_epoch)):
    sum_loss = 0
    for i, batch in tqdm(enumerate(trainloader), total=len(dataset_train)//batch_size):
        model.train()
        image = batch['full_image'].to(device)
        optimizer.zero_grad()
        inpainted_image = model(image)
        loss = criterion(inpainted_image, image)
        sum_loss+=loss.item()
        loss.backward()
        if i%500 ==0 :
            print(f"[Epoch {epoch}/{n_epoch}] [Batch {i}] [loss: {loss.item()}]")
            save_image(image[1,:,:,:], f'./samples/AE_original_{epoch}_{i}.png')
            save_image(inpainted_image[1,:,:,:], f'./samples/AE_recon_{epoch}_{i}.png')
            torch.save(model.state_dict(), f'ckpt_rotfoNET_AE_{epoch}_{i}.pth')
        optimizer.step()
    print('avg epoch loss: ', sum_loss/(len(dataset_train)//batch_size))

[Epoch 0/10] [Batch 0] [loss: 0.3709760308265686]
[Epoch 0/10] [Batch 500] [loss: 0.0410640612244606]
[Epoch 0/10] [Batch 1000] [loss: 0.03879993036389351]
[Epoch 0/10] [Batch 1500] [loss: 0.034863755106925964]
